In [ ]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as goz

import plotly.figure_factory as ff

import warnings
warnings.filterwarnings('ignore')

py.init_notebook_mode(connected=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df=pd.read_csv("ceo_data_pay_merged_r3000.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.head()

### Cleaning Data

In [ ]:
df["industry"] = df["industry"].str.replace(r'[^a-zA-Z_]', ' ', regex=True)
df["industry"] = df["industry"].replace(r'\s+', ' ', regex=True)

df["median_worker_pay"] = df["median_worker_pay"].str.replace(r"[$,]", "", regex=True)
df["median_worker_pay"] = pd.to_numeric(df["median_worker_pay"], errors="coerce")

df["salary"] = df["salary"].str.replace(r"[$,]", "", regex=True)
df["salary"] = pd.to_numeric(df["salary"], errors="coerce")

df["pay_ratio"]=df["pay_ratio"].str.split(':1').str[0]
df["pay_ratio"].replace(',','', regex=True, inplace=True)
df["pay_ratio"] = pd.to_numeric(df["pay_ratio"], errors="coerce")

In [ ]:
df.sort_values(['salary'],ascending=False)

### Feature Engineering

In [ ]:
import yfinance as yf
pd.set_option('display.max_rows', None)

industry=[]
sector=[]
fullTimeEmployees=[]
longBusinessSummary=[]
totalRevenue=[]
grossProfits=[]
freeCashflow=[]
operatingCashflow=[]
for i,ticker_i in enumerate(df["ticker"]):
#     try:
    ticker=yf.Ticker(str(ticker_i))
    print(i,ticker)

    try:
        industry.append(ticker.info["industry"])
    except:
        print("industry-Issue",ticker)
        industry.append("Issue")

    try:
        sector.append(ticker.info["sector"])
    except:
        print("sector-Issue",ticker)
        sector.append("Issue")

    try:
        fullTimeEmployees.append(ticker.info["fullTimeEmployees"])
    except:
        print("fullTimeEmployees-Issue",ticker)
        fullTimeEmployees.append(np.nan)

    try:
        longBusinessSummary.append(ticker.info["longBusinessSummary"])
    except:
        print("longBusinessSummary-Issue",ticker)
        longBusinessSummary.append("Issue")

    try:
        totalRevenue.append(ticker.info["totalRevenue"])
    except:
        print("totalRevenue-Issue",ticker)
        totalRevenue.append(np.nan)

    try:
        grossProfits.append(ticker.info["grossProfits"])
    except:
        print("grossProfits-Issue",ticker)
        grossProfits.append(np.nan)

    try:
        freeCashflow.append(ticker.info["freeCashflow"])
    except:
        print("freeCashflow-Issue",ticker)
        freeCashflow.append(np.nan)

    try:
        operatingCashflow.append(ticker.info["operatingCashflow"])
    except:
        print("operatingCashflow-Issue",ticker)
        operatingCashflow.append(np.nan)
#     except:
#         print("Issue",ticker)
#         industry.append("Issue")
#         sector.append("Issue")
#         fullTimeEmployees.append("Issue")
#         longBusinessSummary.append("Issue")
#         totalRevenue.append("Issue")
#         grossProfits.append("Issue")
#         freeCashflow.append("Issue")
#         operatingCashflow.append("Issue")

In [ ]:
df["industry"]=industry
df["sector"]=sector
df["fullTimeEmployees"]=fullTimeEmployees
df["longBusinessSummary"]=longBusinessSummary
df["totalRevenue"]=totalRevenue
df["grossProfits"]=grossProfits
df["freeCashflow"]=freeCashflow
df["operatingCashflow"]=operatingCashflow

In [ ]:
df.dtypes

In [ ]:
df_before_na = df.copy(deep=True)

In [ ]:
df=df.fillna(0)
# df=df.dropna()

In [ ]:
df

In [ ]:
df.to_csv("Russell3000.csv",index=False)

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly.figure_factory as ff

pd.set_option('display.float_format', lambda x: '%.2f' % x)

df=pd.read_csv("Russell3000.csv")

### Univariate Analysis

In [ ]:
# num_cols=df.select_dtypes(include=[np.number])

num_cols = list(df.select_dtypes(include = np.number).columns)

In [ ]:
colorscale = [[0, '#1f77b4'],[.5, '#aec7e8'],[1, '#ff7f0e']]
fig = ff.create_table(round(df[num_cols].describe().reset_index(), 2), font_colors = ['white'], colorscale = colorscale)

for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 9
    
fig.update_layout(
    title_text = 'Descriptive Statistics on the numerical columns',
    margin = {'t':50},
    template= "plotly_dark"
)
    
fig.show()

In [ ]:
# categorized_cols=df.select_dtypes(exclude=[object])

categorized_cols = list(df.select_dtypes(include = 'object').columns)

In [ ]:
colorscale = [[0, '#1f77b4'],[.5, '#aec7e8'],[1, '#ff7f0e']]
fig = ff.create_table(df[categorized_cols].describe().reset_index(), font_colors = ['white'], colorscale = colorscale)

for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 9
    
fig.update_layout(
    title_text = 'Statistics of the categorical columns',
    margin = {'t':50},
    template= "plotly_dark"
)
    
fig.show()

In [ ]:
num_cols

In [ ]:
def plot_histogram(col_name):
    series = df[col_name]
    # remove zero values items
    series = series[ series != 0 ]
    smin,smax = series.min(),series.max()
    # remove outliers for +- three standard deviations.
    series = series[ ~( ( series - series.mean() ).abs() > 3 * series.std() ) ]
    percentiles = [ np.percentile(series,n) for n in (2.5,50,97.5) ]
    
    trace0 = go.Histogram( x = series,
                            histfunc = 'avg', 
                            histnorm = 'probability density',
                            opacity=.75,
                            marker = {'color':'#EB89B5'})
    data_ = go.Data( [trace0] )
    
    shapes = [{ 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[0], 'x1':percentiles[0], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    { 'line': { 'color': '#00999F', 'dash':'solid', 'width':1 },
                'type':'line',
                'x0':percentiles[1], 'x1':percentiles[1], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    
              { 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[2], 'x1':percentiles[2], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' } 
             ]
    
    annotations = [ {'x': percentiles[0], 'xref':'x','xanchor':'right',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%', 'font':{'size':16},
                     'showarrow':False},
                   
                    {'x': percentiles[1], 'xref':'x','xanchor':'center',
                     'y': .2, 'yref':'paper', 
                     'text':'95%<br>median = {0:,.2f}<br>mean = {1:,.2f}<br>min = {2:,}<br>max = {3:,}'
                         .format(percentiles[1],series.mean(),smin,smax), 
                     'showarrow':False,
                     'font':{'size':20} },
                   
    {'x': percentiles[2], 'xref':'x','xanchor':'left',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%','font':{'size':16}, 
                     'showarrow':False},
                   
                    {'x': .5, 'xref':'paper','xanchor':'center',
                     'y': 1.1, 'yref':'paper','yanchor':'middle', 
                     'text':'Outliers above or below three standard deviations are excluded from the graph, mean and median calculations.',
                     'font':{'size':15,'color':'mistyrose'}, 
                     'showarrow':False} 
                  ]
    
    layout = go.Layout( title = col_name.replace('_',' ').capitalize(),
                        titlefont = {'size':50},
                        yaxis = {'title':'Probability/Density'},
                        xaxis = {'title':col_name, 'type':'linear'},
                        shapes = shapes,
                         annotations = annotations
                         )
    figure = go.Figure(data = data_, layout = layout)
    py.iplot(figure)
    
def plot_histogram_without_outliers(col_name):
    series = df[col_name]
    # remove zero values items
    series = series[ series != 0 ]
    smin,smax = series.min(),series.max()
    # remove outliers for +- three standard deviations.
#     series = series[ ~( ( series - series.mean() ).abs() > 3 * series.std() ) ]
    percentiles = [ np.percentile(series,n) for n in (2.5,50,97.5) ]
    
    trace0 = go.Histogram( x = series,
                            histfunc = 'avg', 
                            histnorm = 'probability density',
                            opacity=.75,
                            marker = {'color':'#EB89B5'})
    data_ = go.Data( [trace0] )
    
    shapes = [{ 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[0], 'x1':percentiles[0], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    { 'line': { 'color': '#00999F', 'dash':'solid', 'width':1 },
                'type':'line',
                'x0':percentiles[1], 'x1':percentiles[1], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' },
    
              { 'line': { 'color': '#0099FF', 'dash':'solid', 'width':2 },
                'type':'line',
                'x0':percentiles[2], 'x1':percentiles[2], 'xref':'x',
                'y0':-0.1, 'y1':1, 'yref':'paper' } 
             ]
    
    annotations = [ {'x': percentiles[0], 'xref':'x','xanchor':'right',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%', 'font':{'size':16},
                     'showarrow':False},
                   
                    {'x': percentiles[1], 'xref':'x','xanchor':'center',
                     'y': .2, 'yref':'paper', 
                     'text':'95%<br>median = {0:,.2f}<br>mean = {1:,.2f}<br>min = {2:,}<br>max = {3:,}'
                         .format(percentiles[1],series.mean(),smin,smax), 
                     'showarrow':False,
                     'font':{'size':20} },
                   
    {'x': percentiles[2], 'xref':'x','xanchor':'left',
                     'y': .3, 'yref':'paper', 
                     'text':'2.5%','font':{'size':16}, 
                     'showarrow':False},
                   
                    {'x': .5, 'xref':'paper','xanchor':'center',
                     'y': 1.1, 'yref':'paper','yanchor':'middle', 
                     'text':'Outliers above or below three standard deviations are excluded from the graph, mean and median calculations.',
                     'font':{'size':15,'color':'mistyrose'}, 
                     'showarrow':False} 
                  ]
    
    layout = go.Layout( title = col_name.replace('_',' ').capitalize(),
                        titlefont = {'size':50},
                        yaxis = {'title':'Probability/Density'},
                        xaxis = {'title':col_name, 'type':'linear'},
                        shapes = shapes,
                         annotations = annotations
                         )
    figure = go.Figure(data = data_, layout = layout)
    py.iplot(figure)
    
def plot_value_counts(col_name, table=False, bar=False):
    N=10
    values_count = pd.DataFrame(df[col_name].value_counts())
    values_count.columns = ['count']
    # convert the index column into a regular column.
    values_count[col_name] = [ str(i) for i in values_count.index ]
    # add a column with the percentage of each data point to the sum of all data points.
    values_count['percent'] = values_count['count'].div(values_count['count'].sum()).multiply(100).round(2)
    # change the order of the columns.
    values_count = values_count.reindex([col_name,'count','percent'],axis=1)
    values_count.reset_index(drop=True,inplace=True)
    values_count=values_count.head(N)
    if bar:
        # add a font size for annotations0 which is relevant to the length of the data points.
        font_size = int(abs(20 - (.25 * len(values_count[col_name]))))
        
        trace0 = go.Bar(x=values_count[col_name], y=values_count['count'])
        data_ = go.Data([trace0])
        
        annotations0 = [dict(x=xi,
                             y=yi, 
                             showarrow=False,
                             font={'size':font_size},
                             text="{:,}".format(yi),
                             xanchor='center',
                             yanchor='bottom')
                        for xi,yi,_ in values_count.values]
        
        annotations1 = [dict(x=xi,
                             y=yi/2,
                             showarrow=False,
                             text="{}%".format(pi),
                             xanchor='center',
                             yanchor='middle',
                             font={'color':'yellow'})
                        for xi,yi,pi in values_count.values if pi > 10]
        
        annotations = annotations0 + annotations1
        
        layout = go.Layout(title=col_name.replace('_',' ').capitalize(),
                           titlefont={'size':50},
                           yaxis={'title':'count'},
                           xaxis={'type':'category'},
                           annotations=annotations)
        figure = go.Figure(data=data_, layout=layout)
        py.iplot(figure)
    
    if table: 
        values_count['count'] = values_count['count'].apply(lambda d : "{:,}".format(d))
        table = ff.create_table(values_count,index_title="race")
        py.iplot(table)
    
    return values_count


In [ ]:
# for col in num_cols :
#     plot_histogram(col)
    
# for col in num_cols :
#     plot_histogram_without_outliers(col)
    



In [ ]:
categorized_cols

In [ ]:
# Remove specific columns from the categorized_cols list
categorized_cols = [col for col in categorized_cols if col not in ['ticker', 'company_name', 'ceo_name', 'longBusinessSummary']]


In [ ]:
df_cat=[]
for col in categorized_cols:
    values_count = plot_value_counts(col,1,1)
    df_cat.append(values_count)

In [ ]:
df

In [ ]:
industry_10=df_cat[0]['industry'].values.tolist()

In [ ]:
df_bi_industry=df[df["industry"].isin(industry_10)]

### Bivariate Analysis

In [ ]:
df

In [ ]:
import plotly.express as px
px.scatter(df, x="median_worker_pay", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="pay_ratio", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="totalRevenue", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="grossProfits", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="freeCashflow", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")

In [ ]:
px.scatter(df, x="operatingCashflow", y="salary",
size="fullTimeEmployees", color="company_name",hover_name="ticker")